In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Configuração visual
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = [10, 6]

# Adicionar pasta src ao path para importar configurações compartilhadas
sys.path.append('../src')
try:
    import config
    print("Configurações importadas com sucesso!")
except ImportError:
    print("ERRO: Certifique-se de que o arquivo config.py está na pasta /src")


## 1. Conexão com o Data Lake (MinIO)
Utilizamos as credenciais definidas no `config.py` para acessar o bucket S3 simulado.


In [ ]:
# Opções de armazenamento para o S3fs (backend do Pandas)
storage_options = {
    "key": config.MINIO_ACCESS_KEY,
    "secret": config.MINIO_SECRET_KEY,
    "client_kwargs": {"endpoint_url": config.MINIO_ENDPOINT}
}

print(f"Conectando ao bucket: {config.BUCKET_NAME}")


## 2. Carregamento dos Dados (Camada Silver)
Vamos ler o arquivo Parquet gerado pelo script de processamento. O formato Parquet é otimizado para leitura e mantém os tipos de dados (datas, números) corretos.


In [ ]:
arquivo_silver = f"s3://{config.BUCKET_NAME}/silver/vendas_tratadas.parquet"

try:
    df = pd.read_parquet(arquivo_silver, storage_options=storage_options)
    print(f"Dados carregados: {df.shape[0]} linhas e {df.shape[1]} colunas.")
    display(df.head())
except Exception as e:
    print(f"Erro ao ler arquivo: {e}")
    print("DICA: Você rodou o script 'src/2_processamento.py' antes?")


## 3. Análise Estatística Descritiva
Verificação rápida da distribuição numérica dos dados.


In [ ]:
# Resumo estatístico das variáveis numéricas
display(df[['preco_unitario', 'quantidade', 'total_venda', 'lucro_bruto', 'avaliacao']].describe())

# Verificar se existem nulos
print("\nValores Nulos por coluna:")
print(df.isnull().sum())


## 4. Visualização: Vendas por Filial e Cidade
Qual filial está performando melhor?


In [ ]:
plt.figure(figsize=(8, 5))
ax = sns.countplot(data=df, x='cidade', palette='viridis', hue='filial')
plt.title('Número de Vendas por Cidade e Filial')
plt.xlabel('Cidade')
plt.ylabel('Contagem de Vendas')
plt.show()

# Faturamento total por cidade
faturamento_cidade = df.groupby('cidade')['total_venda'].sum().reset_index()
plt.figure(figsize=(8, 5))
sns.barplot(data=faturamento_cidade, x='cidade', y='total_venda', palette='magma')
plt.title('Faturamento Total por Cidade ($)')
plt.show()


## 5. Performance por Linha de Produto
Quais categorias de produtos geram mais receita?


In [ ]:
plt.figure(figsize=(12, 6))
order = df.groupby('linha_produto')['total_venda'].sum().sort_values(ascending=False).index
sns.barplot(data=df, x='total_venda', y='linha_produto', estimator=sum, order=order, palette='coolwarm')
plt.title('Faturamento Total por Linha de Produto')
plt.xlabel('Total Vendido ($)')
plt.ylabel('')
plt.show()


## 6. Análise Temporal (Tendência de Vendas)
Como as vendas se comportaram ao longo dos dias disponíveis no dataset?


In [ ]:
# Agrupar vendas por dia
vendas_diarias = df.groupby('data_venda')['total_venda'].sum().reset_index()

plt.figure(figsize=(14, 6))
sns.lineplot(data=vendas_diarias, x='data_venda', y='total_venda', marker='o', color='purple')
plt.title('Evolução Diária das Vendas')
plt.xlabel('Data')
plt.ylabel('Vendas ($)')
plt.grid(True, linestyle='--')
plt.show()


## 7. Correlação e Insights
Existe relação entre a Avaliação do cliente (Rating) e o Lucro Bruto?


In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(data=df, x='avaliacao', y='lucro_bruto', alpha=0.5, color='green')
plt.title('Dispersão: Avaliação x Lucro Bruto')
plt.xlabel('Avaliação do Cliente (0-10)')
plt.ylabel('Lucro Bruto ($)')
plt.show()

print("Correlação entre Avaliação e Total da Venda:", df['avaliacao'].corr(df['total_venda']))
